In [54]:
import numpy as np
import cvxopt as opt
from cvxopt import blas, solvers
import sys
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd
import cufflinks

prices = np.array([
[41.1, 44.2, 51.3, 41.4],
[42.1, 42.2, 58.3, 53.4],
[43.1, 41.2, 57.2, 72.4],
[46.1, 41.2, 53.3, 61.4],
])

print("col 1: ")
print(prices[:,:1])
print(np.var(prices[:,:1])/np.mean(prices[:,:1]))
print(prices[3,0]/prices[0,0])

print("col 2: ")
print(prices[:,1:2])
print(np.var(prices[:,:2])/np.mean(prices[:,:2]))
print(prices[3,1]/prices[0,1])


cov_annual = np.cov(prices)
print(cov_annual)
weights = np.array([0.25,0.25,0.25,0.25])
returns_annual = np.array([0.1, 0.2, 0.3, 0.4])

ret = np.dot(weights, returns_annual)
risk = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
print("risk: ")
print(risk)

%matplotlib inline



return_vec = prices 

def random_portfolio(returns):
    
    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # This recursion reduces outliers to keep plots pretty
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma


n_portfolios = 6


def optimal_portfolio(returns):
    print("returns: ")
    print(returns)
    
    ret = returns[1:]/returns[:-1]
    
    arr = np.ones((1,4))
    arr = np.append(arr, ret, axis=0)
    #returns = arr
    returns = arr - np.ones((4,4))
    print("ret: ")
    print(arr)
    
    n = len(returns)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    print("cov: ")
    print(np.cov(returns))
    print("S:")
    print(S)
    pbar = opt.matrix(np.mean(returns, axis=1))
    print("pbar: ")
    print(pbar) #
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    print("about to return..")
    return np.asarray(wt), returns, risks

######
weights, returns, risks = optimal_portfolio(return_vec)
print("optimal:")
print(weights)

col 1: 
[[41.1]
 [42.1]
 [43.1]
 [46.1]]
0.08120649651972157
1.121654501216545
col 2: 
[[44.2]
 [42.2]
 [41.2]
 [41.2]]
0.06336459554513481
0.9321266968325792
[[ 22.5         25.03333333   1.87333333   1.        ]
 [ 25.03333333  66.56666667  90.99        55.86666667]
 [  1.87333333  90.99       210.11583333 125.50666667]
 [  1.          55.86666667 125.50666667  77.5       ]]
risk: 
7.815139554096265
returns: 
[[41.1 44.2 51.3 41.4]
 [42.1 42.2 58.3 53.4]
 [43.1 41.2 57.2 72.4]
 [46.1 41.2 53.3 61.4]]
ret: 
[[1.         1.         1.         1.        ]
 [1.0243309  0.95475113 1.13645224 1.28985507]
 [1.02375297 0.97630332 0.98113208 1.35580524]
 [1.06960557 1.         0.93181818 0.8480663 ]]
cov: 
[[ 0.          0.          0.          0.        ]
 [ 0.          0.0213965   0.02268472 -0.01213166]
 [ 0.          0.02268472  0.03322931 -0.01281293]
 [ 0.         -0.01213166 -0.01281293  0.00897141]]
S:
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  2.14e-02  2.27e-02 -1.21e-02]